In [1]:
import pandas as pd
import numpy as np
import csv
# Reading Files
emp= pd.read_csv("train.csv")
emp2= pd.read_csv("test.csv")

In [2]:
# removing unwanted features
emp = emp.drop(["ID","FormalWarnings","EmployeeCount","Behaviour","StandardHours","Over18"], axis=1)
emp2 = emp2.drop(["ID","FormalWarnings","EmployeeCount","Behaviour","StandardHours","Over18"], axis=1)

In [3]:
# Converting categorical variable into dummy/indicator variables
emp1 = pd.get_dummies(emp, drop_first=True)
emp2 = pd.get_dummies(emp2, drop_first=True)

In [4]:
emp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                1229 non-null   float64
 1   DailyRate                          1229 non-null   float64
 2   DistanceFromHome                   1230 non-null   int64  
 3   HourlyRate                         1228 non-null   float64
 4   JobInvolvement                     1229 non-null   float64
 5   JobLevel                           1230 non-null   int64  
 6   JobSatisfaction                    1229 non-null   float64
 7   MonthlyIncome                      1230 non-null   int64  
 8   MonthlyRate                        1229 non-null   float64
 9   NumCompaniesWorked                 1229 non-null   float64
 10  PercentSalaryHike                  1229 non-null   float64
 11  PerformanceRating                  1230 non-null   int64

In [5]:
# Corelating Other Features with Attrition
corr_matrix = emp1.corr()
corr_matrix['Attrition'].sort_values(ascending=False)

Attrition                            1.000000
OverTime_Yes                         0.234594
MaritalStatus_Single                 0.165029
JobRole_Sales Representative         0.148548
BusinessTravel_Travel_Frequently     0.146434
JobRole_Laboratory Technician        0.097545
Department_Sales                     0.082222
DistanceFromHome                     0.075607
EducationField_Technical Degree      0.059178
EducationField_Marketing             0.056283
NumCompaniesWorked                   0.042284
LvlGrownInOrg                        0.040906
JobRole_Human Resources              0.040152
Gender_Male                          0.024812
JobRole_Sales Executive              0.022839
JobRole_Research Scientist           0.006765
Experience                          -0.001739
MonthlyRate                         -0.002527
EmployeeNumber                      -0.009269
PerformanceRating                   -0.009508
EducationField_Other                -0.014464
HourlyRate                        

In [6]:
# creating pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
])

In [7]:
emp1.shape

(1230, 52)

In [8]:
emp1

,Age,DailyRate,DistanceFromHome,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,...,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree
0,44.0,1376.0,1,91.0,2.0,3,1.0,10482,2326.0,9.0,...,0,0,1,0,0,0,0,1,0,0
1,25.0,675.0,5,85.0,4.0,2,1.0,4000,18384.0,1.0,...,0,0,0,0,0,1,0,0,0,0
2,57.0,210.0,29,56.0,2.0,4,4.0,14118,22102.0,3.0,...,0,0,0,0,0,0,1,0,0,0
3,47.0,1180.0,25,84.0,3.0,3,3.0,8633,13084.0,2.0,...,0,0,0,1,0,0,0,1,0,0
4,22.0,391.0,7,75.0,3.0,1,2.0,2472,26092.0,1.0,...,0,0,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,29.0,806.0,7,39.0,3.0,1,3.0,3339,17285.0,3.0,...,0,0,0,0,1,0,0,0,0,1
1226,41.0,582.0,28,60.0,2.0,4,2.0,13570,5640.0,0.0,...,0,0,1,0,0,1,0,0,0,0
1227,36.0,1456.0,13,96.0,2.0,2,1.0,6134,8658.0,5.0,...,1,0,0,0,1,0,1,0,0,0
1228,23.0,885.0,4,58.0,4.0,1,1.0,2819,8544.0,2.0,...,0,0,1,0,0,0,0,1,0,0


In [9]:
# Splitting Features and Labels
emp_train_labels_C1 = emp1[["Attrition"]].copy()
emp_train_labels_R1 = emp1[["YearsAtCompany"]].copy()
emp_train_features = emp1.drop(["YearsAtCompany","Attrition"], axis=1)

In [10]:
emp_train_labels_C1.shape

(1230, 1)

In [11]:
emp_train_labels_C = my_pipeline.fit_transform(emp_train_labels_C1)
emp_train_labels_R = my_pipeline.fit_transform(emp_train_labels_R1)
emp_features = my_pipeline.fit_transform(emp_train_features)
emp_test = my_pipeline.fit_transform(emp2)

In [12]:
emp_train_labels_C.shape

(1230, 1)

In [13]:
# Selecting Model

# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.ensemble import ExtraTreesRegressor
# model = KNeighborsRegressor()
# model = ExtraTreesRegressor(n_estimators=10,max_features=32,random_state=0)
# model = LinearRegression()
# model = DecisionTreeRegressor()
model1 = RandomForestRegressor(max_depth=30,random_state=0)
model1.fit(emp_features, emp_train_labels_R)

<ipython-input-13-a76708fd9090>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model1.fit(emp_features, emp_train_labels_R)


RandomForestRegressor(max_depth=30, random_state=0)

In [14]:
emp_train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 50 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                1229 non-null   float64
 1   DailyRate                          1229 non-null   float64
 2   DistanceFromHome                   1230 non-null   int64  
 3   HourlyRate                         1228 non-null   float64
 4   JobInvolvement                     1229 non-null   float64
 5   JobLevel                           1230 non-null   int64  
 6   JobSatisfaction                    1229 non-null   float64
 7   MonthlyIncome                      1230 non-null   int64  
 8   MonthlyRate                        1229 non-null   float64
 9   NumCompaniesWorked                 1229 non-null   float64
 10  PercentSalaryHike                  1229 non-null   float64
 11  PerformanceRating                  1230 non-null   int64

In [15]:
# print(emp_train_labels_C)
emp_train_labels_C.shape
print(emp_features.shape)

(1230, 50)


In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
# model = make_pipeline(SVC(gamma='auto'))
# model = GaussianNB()
# model = DecisionTreeClassifier(criterion="entropy",max_depth=4,random_state=42)
model = RandomForestClassifier(criterion="entropy",max_depth=4,random_state=42)
# model = KNeighborsClassifier(n_neighbors=3)
model.fit(emp_features, emp_train_labels_C)

<ipython-input-16-7066a0963a31>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(emp_features, emp_train_labels_C)


RandomForestClassifier(criterion='entropy', max_depth=4, random_state=42)

In [17]:
predictions=model.predict(emp_features)
print(predictions)

[0. 0. 0. ... 0. 0. 0.]


In [18]:
predictions1=model1.predict(emp_features)
print(predictions1)

[18.05  5.78  0.91 ...  2.19  2.87 10.  ]


In [19]:
final_prediction_proba = model.predict_proba(emp_test)
print(final_prediction_proba)

[[0.81547205 0.18452795]
 [0.74939181 0.25060819]
 [0.90123256 0.09876744]
 [0.80132759 0.19867241]
 [0.84088144 0.15911856]
 [0.72903247 0.27096753]
 [0.74924929 0.25075071]
 [0.87457247 0.12542753]
 [0.92419765 0.07580235]
 [0.86346837 0.13653163]
 [0.80901627 0.19098373]
 [0.74232427 0.25767573]
 [0.83728646 0.16271354]
 [0.67357881 0.32642119]
 [0.94492849 0.05507151]
 [0.85602829 0.14397171]
 [0.7993636  0.2006364 ]
 [0.88293591 0.11706409]
 [0.88322322 0.11677678]
 [0.85116387 0.14883613]
 [0.92116546 0.07883454]
 [0.79410661 0.20589339]
 [0.56683057 0.43316943]
 [0.76477107 0.23522893]
 [0.8993866  0.1006134 ]
 [0.8243865  0.1756135 ]
 [0.86274482 0.13725518]
 [0.89642064 0.10357936]
 [0.8379888  0.1620112 ]
 [0.87694291 0.12305709]
 [0.71770549 0.28229451]
 [0.69143771 0.30856229]
 [0.73762639 0.26237361]
 [0.80963175 0.19036825]
 [0.91144611 0.08855389]
 [0.65368953 0.34631047]
 [0.8490797  0.1509203 ]
 [0.93210403 0.06789597]
 [0.86496707 0.13503293]
 [0.92056904 0.07943096]


In [20]:
# final_prediction = model.predict_proba(emp_test)
# print(final_prediction)

In [21]:
final_prediction_test = model.predict_proba([emp_features[2]])
print(final_prediction_test)

[[0.85731213 0.14268787]]


In [22]:
final_prediction1 = model1.predict(emp_test)
print(final_prediction1)

[ 2.87  3.55  2.94  6.05  3.78  1.77  6.12  4.51 15.01  2.9   2.25  4.94
  2.44  0.79 14.87  4.72  2.38  3.23  3.33  4.78 12.45  8.43  1.13  4.03
  0.74  3.03  7.71  6.4   2.39  4.92  5.34  0.8   0.68  2.42 13.18  1.37
 12.96  4.74  2.96  6.54 13.61 13.09  8.35 31.9   2.94  0.94  3.39  1.1
  0.84  5.26  9.41  9.23  7.54  2.79  7.46  0.22 32.9  11.25  5.19 21.6
  9.95  9.97  9.27  4.16 19.31  0.9  11.02 10.08 10.92  2.49 16.39  0.92
 24.32  7.71  4.59  4.6   0.94 19.83  3.23  4.72  7.5   4.88  2.85 19.16
 17.27  1.93  9.3   3.09 21.56 17.25 16.51  7.78  4.67  2.15 10.1   0.83
  4.83 32.57  0.98 11.15  7.26 10.41 13.28  9.79  4.42  2.47 31.88  0.91
  5.35 13.68  7.9   4.75 10.14 11.84 11.98  7.76 13.86 10.1   3.21  2.75
  2.34  0.76 15.49 21.52  8.11  0.77  6.12  8.2   0.84  5.18  0.82  1.82
 11.   13.02  4.86 10.55 14.35  5.45 24.47  2.47  9.    9.49  1.59  1.14
 20.29 13.21  9.5   2.88  5.43 14.47  5.51  4.79 18.43 11.85 10.58  9.4
  2.93  9.37  5.04  3.1   5.92  0.67  8.68  3.52  5.79

In [23]:
# Calculating Root Mean Squared Error
from sklearn.metrics import accuracy_score
print(accuracy_score(emp_train_labels_C,predictions))

0.8569105691056911


In [24]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, emp_features, emp_train_labels_C, cv=10)
print(scores.mean())

c:\users\nishi.suraj-pc\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\nishi.suraj-pc\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\nishi.suraj-pc\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\users\nishi.s

0.8487804878048781


In [25]:
# Exporting to .csv file 
with open('Final_Predictions_without_round.csv', 'w', newline='') as file:
    fieldnames = ['ID','Attrition', 'YearsAtCompany']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(final_prediction)):
        writer.writerow({'ID': i+1, 'Attrition': final_prediction[i][1],'YearsAtCompany':final_prediction1[i]})